In [1]:
import cv2
import time
import numpy as np
import contextlib
import torch.cuda.amp
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import scipy.io

import spconv.pytorch as spconv
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F

from util.BoundingBoxRegressionUtils import parse_random_dataset, parse_partial_dataset, torch_iou

Event-based vision module imported


In [2]:
def getData(filename):
    img = load_img(filename)
    
    data = img_to_array(img)
    arr = np.array(data)
    width = arr.shape[1]
    height = arr.shape[0]
    
    arr = cv2.resize(arr, (128,128)) / 256
    
    return width, height, arr

def getAnnotation(filename):
    mat = scipy.io.loadmat(filename)
    data = np.array(mat["box_coord"][0])
    minX = data[2]
    maxX = data[3]
    minY = data[0]
    maxY = data[1]
    
    return minX, minY, maxX, maxY

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    amp_ctx = contextlib.nullcontext()
    if args.fp16:
        amp_ctx = torch.cuda.amp.autocast()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        with amp_ctx:
            output = model(data)
            loss = F.mse_loss(output, target)
            loss.backward()
            optimizer.step()

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    amp_ctx = contextlib.nullcontext()
    if args.fp16:
        amp_ctx = torch.cuda.amp.autocast()

    with torch.no_grad():
        for data, target in test_loader:

            data, target = data.to(device), target.to(device)
            with amp_ctx:

                output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            
            correct += torch.sum(torch_iou(output[:, -4:], target[:, -4:]))
            

    test_loss /= len(test_loader.dataset)

    
    return correct / len(test_loader.dataset)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = spconv.SparseSequential(
            spconv.SparseConv2d(3, 16, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(16, 32, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(32, 64, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(64, 128, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(128, 256, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.ToDense(),
            nn.Flatten(1,-1),
            nn.Dropout(p=0.2),
            nn.Linear(4096, 2048),
            nn.Linear(2048, 1024),
            nn.Linear(1024, 8),
        )

    def forward(self, x: torch.Tensor):
        x = spconv.SparseConvTensor.from_dense(x.reshape(-1, 128, 128, 3))
        y =  self.net(x)
        
        return y

In [5]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation, False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_loader = torch.utils.data.DataLoader(
    [[X_train[i], y_train[i]] for i in range(len(X_train))], shuffle=False, batch_size=40)
# test_loader = torch.utils.data.DataLoader(
#     [[X_test[i], y_test[i]] for i in range(len(X_test))], shuffle=False, batch_size=40)

args = lambda: None
args.fp16 = True
args.lr = 0.001
args.log_interval = 50

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

def makeTestData():
    testData = []
    
    for i in range(10): 
        X_test, y_test = parse_random_dataset(getData, getAnnotation, False)
        test_loader = torch.utils.data.DataLoader(
            [[X_test[i], y_test[i]] for i in range(len(X_test))], shuffle=False, batch_size=40)
        
        testData.append(test_loader)
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_loader = testData[i]

        testing_time = time.time()

        iou = test(args, model, device, test_loader)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    
    train_start_time = time.process_time()

    for epoch in range(0, 40):
        train(args, model, device, train_loader, optimizer, epoch)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

cuda:0
Round  0
Training done!  439.625
	For this mini round, time:  1.1820001602172852  iou  tensor(0.7090, device='cuda:0')
	For this mini round, time:  1.1450011730194092  iou  tensor(0.7187, device='cuda:0')
	For this mini round, time:  1.1629998683929443  iou  tensor(0.7034, device='cuda:0')
	For this mini round, time:  1.218001127243042  iou  tensor(0.7097, device='cuda:0')
	For this mini round, time:  1.1969990730285645  iou  tensor(0.7055, device='cuda:0')
	For this mini round, time:  1.150998592376709  iou  tensor(0.7141, device='cuda:0')
	For this mini round, time:  1.1700000762939453  iou  tensor(0.7049, device='cuda:0')
	For this mini round, time:  1.1409990787506104  iou  tensor(0.7180, device='cuda:0')
	For this mini round, time:  1.1660058498382568  iou  tensor(0.7162, device='cuda:0')
	For this mini round, time:  1.1990010738372803  iou  tensor(0.7021, device='cuda:0')
Time on avg:  1.1732006072998047  IOU on avg:  tensor(0.7102, device='cuda:0')
Round  1
Training done!

	For this mini round, time:  1.1369988918304443  iou  tensor(0.7825, device='cuda:0')
	For this mini round, time:  1.1330010890960693  iou  tensor(0.7749, device='cuda:0')
	For this mini round, time:  1.13100004196167  iou  tensor(0.7811, device='cuda:0')
	For this mini round, time:  1.1359984874725342  iou  tensor(0.7791, device='cuda:0')
	For this mini round, time:  1.1459991931915283  iou  tensor(0.7684, device='cuda:0')
Time on avg:  1.1327998161315918  IOU on avg:  tensor(0.7767, device='cuda:0')
Round  9
Training done!  416.9375
	For this mini round, time:  1.1389999389648438  iou  tensor(0.6844, device='cuda:0')
	For this mini round, time:  1.1189994812011719  iou  tensor(0.6970, device='cuda:0')
	For this mini round, time:  1.1219995021820068  iou  tensor(0.6776, device='cuda:0')
	For this mini round, time:  1.115999460220337  iou  tensor(0.6862, device='cuda:0')
	For this mini round, time:  1.125000238418579  iou  tensor(0.6814, device='cuda:0')
	For this mini round, time:  1.

Training done!  419.078125
	For this mini round, time:  1.1329994201660156  iou  tensor(0.7827, device='cuda:0')
	For this mini round, time:  1.122997760772705  iou  tensor(0.7918, device='cuda:0')
	For this mini round, time:  1.1200006008148193  iou  tensor(0.7833, device='cuda:0')
	For this mini round, time:  1.1219980716705322  iou  tensor(0.7852, device='cuda:0')
	For this mini round, time:  1.1679999828338623  iou  tensor(0.7828, device='cuda:0')
	For this mini round, time:  1.1709980964660645  iou  tensor(0.7886, device='cuda:0')
	For this mini round, time:  1.1259992122650146  iou  tensor(0.7872, device='cuda:0')
	For this mini round, time:  1.1169993877410889  iou  tensor(0.7875, device='cuda:0')
	For this mini round, time:  1.1250007152557373  iou  tensor(0.7883, device='cuda:0')
	For this mini round, time:  1.1170001029968262  iou  tensor(0.7806, device='cuda:0')
Time on avg:  1.1321993350982666  IOU on avg:  tensor(0.7858, device='cuda:0')
Round  18
Training done!  424.26562

(array([439.625   , 423.625   , 424.109375, 421.921875, 418.125   ,
        417.640625, 416.234375, 415.34375 , 425.625   , 416.9375  ,
        422.171875, 422.1875  , 423.46875 , 423.078125, 423.6875  ,
        423.65625 , 423.546875, 419.078125, 424.265625, 420.78125 ]),
 array([1.17320061, 1.12870002, 1.1275008 , 1.13630023, 1.16820042,
        1.19040034, 1.12610004, 1.13200009, 1.13279982, 1.12219989,
        1.12289989, 1.12070019, 1.12010071, 1.11610031, 1.11820006,
        1.11620045, 1.11920037, 1.13219934, 1.12090032, 1.12110007]),
 array([0.71016449, 0.75654006, 0.72461247, 0.72426671, 0.69404417,
        0.74259561, 0.74962711, 0.76263976, 0.77672899, 0.68542016,
        0.77729839, 0.74367559, 0.67529362, 0.6816051 , 0.69481021,
        0.67986947, 0.69440752, 0.78580296, 0.69467705, 0.74316496]))